In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
# Import required libraries

import torch
import torch.nn as nn
import torch.optim as optim
import torchinfo
import torchvision
import torchvision.models as models
from torch import nn
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from scipy.io import loadmat
import matplotlib.pyplot as plt
import random
from pathlib import Path
from PIL import Image
from tqdm import tqdm
import datetime
import time
import numpy as np 
import pandas as pd 

In [2]:
class CarsDataset(Dataset):
    def __init__(self, annotations, img_dir, transform=None):
        self.img_paths = [os.path.join(img_dir, sample[-1][0]) for sample in annotations[0]]
        self.labels = [sample[-2][0][0] - 1 for sample in annotations[0]]
        self.transform = transform

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        image = Image.open(self.img_paths[idx]).convert("RGB")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)
        
        return image, label

cars_dataset = torch.load("cars196_dataset.pt", weights_only=False)

In [3]:
# הצגת גודל הדאטהסט
print(f"מספר דוגמאות בדאטהסט: {len(cars_dataset)}")


מספר דוגמאות בדאטהסט: 16185


In [10]:
class CarsDatasetWrapper(Dataset):
    def __init__(self, subset, transform=None):
        self.subset = subset
        self.transform = transform

    def __len__(self):
        return len(self.subset)

    def __getitem__(self, idx):
        image, label = self.subset[idx]  # שליפת נתונים מה-Subset
        if self.transform:
            if isinstance(image, torch.Tensor):  # **בדיקה אם התמונה כבר טנסור**
                pass  # אם זה כבר טנסור, לא צריך להפעיל שוב את `ToTensor()`
            else:
                image = self.transform(image)  # **הוספת האוגמנטציה**
        return image, label

In [12]:
# הגדרת טרנספורמציות לתמונות
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # שינוי גודל כל התמונות ל-224x224 פיקסלים
    transforms.RandomHorizontalFlip(p=0.5),  # היפוך אופקי של התמונה בהסתברות של 50%
    transforms.RandomRotation(15),  # סיבוב אקראי של התמונה עד 15 מעלות לכל כיוון
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),  # שינוי אקראי של הבהירות, הניגודיות, הרווייה והגוון של התמונה
    transforms.RandomAffine(degrees=0, translate=(0.2, 0.2)), # תזוזה אקראית של התמונה עד 20% מהגודל שלה (לרוחב ולאורך)
    transforms.RandomPerspective(distortion_scale=0.3, p=0.5),  # עיוות פרספקטיבה בהסתברות של 50%, עם עיוות מקסימלי של 30%
    transforms.RandomResizedCrop(224, scale=(0.7, 1.0)),  # חיתוך אקראי מהתמונה תוך שמירה על יחס גובה-רוחב, עם קנה מידה בין 70% ל-100% מהגודל המקורי
    transforms.ToTensor(),  # המרת התמונה ל- **טנסור של PyTorch** (C, H, W) עם טווח ערכים בין 0 ל-1
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # נרמול התמונה לערכים עם ממוצע וסטיית תקן כמו של **ImageNet**, לשיפור האימון
])


test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [14]:
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset

# חלוקת הדאטה לאימון ולמבחן
dataset_size = len(cars_dataset)
train_size = int(0.7 * dataset_size)  # 70% מהדאטה
test_size = dataset_size - train_size  # 30% הנותרים

train_indices, test_indices = train_test_split(range(dataset_size), train_size=train_size, test_size=test_size, random_state=42)

# יצירת סטים עם אוגמנטציה
train_dataset = Subset(cars_dataset, train_indices)
test_dataset = Subset(cars_dataset, test_indices)

# עטיפת ה-Subset עם הטרנספורמציות
train_dataset = CarsDatasetWrapper(train_dataset, transform=train_transform)
test_dataset = CarsDatasetWrapper(test_dataset, transform=test_transform)

print(f"Training samples: {len(train_dataset)}, Testing samples: {len(test_dataset)}")

# יצירת DataLoader עם סטים מעודכנים
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

Training samples: 11329, Testing samples: 4856


In [16]:
from torchvision.models import ResNet50_Weights

base_model = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)


In [18]:
class CustomMobileNet(nn.Module):
    def __init__(self, base_model, num_classes=196):
        super(CustomMobileNet, self).__init__()
        self.base = nn.Sequential(*list(base_model.children())[:-2])  # שמירה על כל השכבות של 
        self.global_avg_pool = nn.AdaptiveAvgPool2d(1)  # pooling
        self.dropout = nn.Dropout(0.5)  # הוסף שכבת Dropout
        self.fc = nn.Linear(2048, num_classes)

    def forward(self, x):
        x = self.base(x)
        x = self.global_avg_pool(x)
        x = torch.flatten(x, 1)  # Flatten לפני fully connected layers
        x = self.dropout(x)  # Dropout לפני השכבה הסופית
        x = self.fc(x)
        return x

# יצירת המודל עם 196 מחלקות
model = CustomMobileNet(base_model, num_classes=196)

In [20]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
model.to(device)

CustomMobileNet(
  (base): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Conv2d

In [22]:
dummy_input = torch.randn(1, 3, 224, 224)  # תמונה מדומה
output = model(dummy_input)
print("Output shape:", output.shape)  # צריך להיות torch.Size([1, 196])


Output shape: torch.Size([1, 196])


In [24]:
from torchinfo import summary

summary(model, input_size=(1, 3, 224, 224), col_names=["input_size", "output_size", "num_params", "trainable"])


Layer (type:depth-idx)                        Input Shape               Output Shape              Param #                   Trainable
CustomMobileNet                               [1, 3, 224, 224]          [1, 196]                  --                        True
├─Sequential: 1-1                             [1, 3, 224, 224]          [1, 2048, 7, 7]           --                        True
│    └─Conv2d: 2-1                            [1, 3, 224, 224]          [1, 64, 112, 112]         9,408                     True
│    └─BatchNorm2d: 2-2                       [1, 64, 112, 112]         [1, 64, 112, 112]         128                       True
│    └─ReLU: 2-3                              [1, 64, 112, 112]         [1, 64, 112, 112]         --                        --
│    └─MaxPool2d: 2-4                         [1, 64, 112, 112]         [1, 64, 56, 56]           --                        --
│    └─Sequential: 2-5                        [1, 64, 56, 56]           [1, 256, 56, 56]        

In [26]:
print(f"Train Loader Size: {len(train_loader.dataset)}")
print(f"Test Loader Size: {len(test_loader.dataset)}")


Train Loader Size: 11329
Test Loader Size: 4856


In [28]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=3, threshold=0.9)

In [30]:
def train_epoch(model, train_loader, criterion, optimizer, scheduler):
    model.train()
    running_loss = 0
    
    print("Starting Training Epoch...")

    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        target = target.to(device).long()
        outputs = model(data)
        loss = criterion(outputs, target)
        running_loss += loss.item()
        loss.backward()
        optimizer.step()

    running_loss /= len(train_loader)
    
    print('Train Loss:', running_loss)
    return running_loss


def test_model(model, test_loader, criterion):
    with torch.no_grad():
        model.eval()
        running_loss = 0
        total_predictions = 0
        correct_predictions = 0

        for batch_idx, (data, target) in enumerate(test_loader):
            data = data.to(device)
            target = target.to(device).long()
            outputs = model(data)  # forward step
            _, predicted = torch.max(outputs.data, 1)
            total_predictions += target.size(0)
            correct_predictions += (predicted == target).sum().item()
            
            loss = criterion(outputs, target)  # loss
            running_loss += loss.item()
            
        running_loss /= len(test_loader)
        acc = (correct_predictions / total_predictions) * 100.0
        print('Test Loss:', running_loss, 'Accuracy:', acc, '%')
        return running_loss, acc

In [32]:
layers_to_train = 8
for param in model.parameters():
    param.requires_grad = False  # מקפיא את כל השכבות

for param in list(model.base.parameters())[-layers_to_train:]:  
    param.requires_grad = True  # פותח השכבות האחרונות לאימון

In [34]:
n_epochs = 15
Train_loss = []
Test_loss = []
Test_acc = []

try:
    for i in range(n_epochs):
        train_loss = train_epoch(model, train_loader, criterion, optimizer, scheduler)  
        test_loss, test_acc = test_model(model, test_loader, criterion)
        
        Train_loss.append(train_loss)
        Test_loss.append(test_loss)
        Test_acc.append(test_acc)
        
        print('*' * 60)

except Exception as e:
    print("Error during training:", str(e))


Starting Training Epoch...
Train Loss: 4.897201775832915
Test Loss: 4.387911851468839 Accuracy: 15.074135090609555 %
************************************************************
Starting Training Epoch...
Train Loss: 4.239113963489801
Test Loss: 3.968842498565975 Accuracy: 24.6499176276771 %
************************************************************
Starting Training Epoch...
Train Loss: 3.894526634081988
Test Loss: 3.7164823632491264 Accuracy: 32.37232289950577 %
************************************************************
Starting Training Epoch...
Train Loss: 3.6127395005293295
Test Loss: 3.5280948626367667 Accuracy: 35.728995057660626 %
************************************************************
Starting Training Epoch...
Train Loss: 3.380571442590633
Test Loss: 3.330785975644463 Accuracy: 39.90939044481054 %
************************************************************
Starting Training Epoch...
Train Loss: 3.146136897046801
Test Loss: 3.171225632491865 Accuracy: 41.371499176276

In [35]:
layers_to_train = 10
for param in model.parameters():
    param.requires_grad = False  # מקפיא את כל השכבות

for param in list(model.base.parameters())[-layers_to_train:]:  
    param.requires_grad = True  # פותח השכבות האחרונות לאימון

In [36]:
n_epochs = 15
Train_loss = []
Test_loss = []
Test_acc = []

try:
    for i in range(n_epochs):
        train_loss = train_epoch(model, train_loader, criterion, optimizer, scheduler)  
        test_loss, test_acc = test_model(model, test_loader, criterion)
        
        Train_loss.append(train_loss)
        Test_loss.append(test_loss)
        Test_acc.append(test_acc)
        
        print('*' * 60)

except Exception as e:
    print("Error during training:", str(e))

Starting Training Epoch...
Train Loss: 1.6324496685619085
Test Loss: 2.135765147836585 Accuracy: 53.95387149917627 %
************************************************************
Starting Training Epoch...
Train Loss: 1.1515196989959395
Test Loss: 2.0455606760163056 Accuracy: 56.610378912685334 %
************************************************************
Starting Training Epoch...
Train Loss: 0.8661039497650845
Test Loss: 2.035429734932749 Accuracy: 56.05436573311368 %
************************************************************
Starting Training Epoch...
Train Loss: 0.6570340656898391
Test Loss: 1.9902351686829014 Accuracy: 59.65815485996705 %
************************************************************
Starting Training Epoch...
Train Loss: 0.4899776259778251
Test Loss: 1.850536676613908 Accuracy: 61.28500823723228 %
************************************************************
Starting Training Epoch...
Train Loss: 0.37628881318468443
Test Loss: 1.814874593364565 Accuracy: 62.252883

In [ ]:
##########

In [39]:
torch.save(model.state_dict(), "test_18.03.pth")
print("Model saved successfully.")

Model saved successfully.


In [42]:
n_epochs = 10
try:
    for i in range(n_epochs):
        train_loss = train_epoch(model, train_loader, criterion, optimizer, scheduler)  
        test_loss, test_acc = test_model(model, test_loader, criterion)
        
        Train_loss.append(train_loss)
        Test_loss.append(test_loss)
        Test_acc.append(test_acc)
        
        print('*' * 60)

except Exception as e:
    print("Error during training:", str(e))

Starting Training Epoch...
Train Loss: 0.06971881735912511
Test Loss: 1.721673428228027 Accuracy: 67.5658978583196 %
************************************************************
Starting Training Epoch...
Train Loss: 0.06872709143224737
Test Loss: 1.842686434325419 Accuracy: 64.39456342668863 %
************************************************************
Starting Training Epoch...


KeyboardInterrupt: 

In [ ]:
layers_to_train = 12
for param in model.parameters():
    param.requires_grad = False  # מקפיא את כל השכבות

for param in list(model.base.parameters())[-layers_to_train:]:  
    param.requires_grad = True  # פותח השכבות האחרונות לאימון

In [ ]:
n_epochs = 20
Train_loss = []
Test_loss = []
Test_acc = []

try:
    for i in range(n_epochs):
        train_loss = train_epoch(model, train_loader, criterion, optimizer, scheduler)  
        test_loss, test_acc = test_model(model, test_loader, criterion)
        
        Train_loss.append(train_loss)
        Test_loss.append(test_loss)
        Test_acc.append(test_acc)
        
        print('*' * 60)

except Exception as e:
    print("Error during training:", str(e))